# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

In [1]:
import os

def get_files(path):
    all_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            all_files.append(os.path.join(root, file))
    return all_files

In [2]:
import pandas as pd

In [3]:
song_files = get_files('../data/song_data')

dfs = []
for file in song_files:
    df = pd.read_json(file, lines=True)
    dfs.append(df)
df1 = pd.concat(dfs, ignore_index=True)
df1

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
...,...,...,...,...,...,...,...,...,...,...
66,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008
67,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
68,1,ARVBRGZ1187FB4675A,NaN,NaN,,Gwen Stefani,SORRZGD12A6310DBC3,Harajuku Girls,290.55955,2004
69,1,ARWB3G61187FB49404,NaN,NaN,"Hamilton, Ohio",Steve Morse,SODAUVL12A8C13D184,Prognosis,363.85914,2000


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_songs         71 non-null     int64  
 1   artist_id         71 non-null     object 
 2   artist_latitude   31 non-null     float64
 3   artist_longitude  31 non-null     float64
 4   artist_location   71 non-null     object 
 5   artist_name       71 non-null     object 
 6   song_id           71 non-null     object 
 7   title             71 non-null     object 
 8   duration          71 non-null     float64
 9   year              71 non-null     int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 5.7+ KB


### #1: `songs` Table

In [5]:
df1.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [6]:
songs_df = df1[['song_id', 'title', 'artist_id', 'year', 'duration']]
songs_df

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
1,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546
2,SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,0,177.47546
3,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
4,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608
...,...,...,...,...,...
66,SOINLJW12A8C13314C,City Slickers,AR8IEZO1187B99055E,2008,149.86404
67,SOGDBUF12A8C140FAA,Intro,AR558FS1187FB45658,2003,75.67628
68,SORRZGD12A6310DBC3,Harajuku Girls,ARVBRGZ1187FB4675A,2004,290.55955
69,SODAUVL12A8C13D184,Prognosis,ARWB3G61187FB49404,2000,363.85914


In [7]:
import configparser

config = configparser.ConfigParser()
config.read('../psqlconfig.ini')

host = config['postgres']['host']
port = config['postgres']['port']
user = config['postgres']['user']
passwd = config['postgres']['passwd']
db = config['postgres']['db']

In [8]:
import psycopg2

conn_string = f'dbname={db} user={user} password={passwd} host={host} port={port}'
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [9]:
for i, row in songs_df.iterrows():
    sql = """
    INSERT INTO songs VALUES (%s, %s, %s, %s, %s)
    """
    cur.execute(sql, (row['song_id'], row['title'], row['artist_id'], row['year'], row['duration']))
    conn.commit() # Make the changes to the database

### #2: `artists` Table

In [10]:
df1.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [11]:
artists_df = df1[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artists_df = artists_df.drop_duplicates(subset=['artist_id'])
artists_df

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
1,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892
2,ARKRRTF1187B9984DA,Sonora Santanera,,NaN,NaN
3,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
4,ARXR32B1187FB57099,Gob,,NaN,NaN
...,...,...,...,...,...
66,AR8IEZO1187B99055E,Marc Shaiman,,NaN,NaN
67,AR558FS1187FB45658,40 Grit,,NaN,NaN
68,ARVBRGZ1187FB4675A,Gwen Stefani,,NaN,NaN
69,ARWB3G61187FB49404,Steve Morse,"Hamilton, Ohio",NaN,NaN


In [12]:
for i, row in artists_df.iterrows():
    sql = """
    INSERT INTO artists VALUES (%s, %s, %s, %s, %s)
    """
    cur.execute(sql, (row['artist_id'], row['artist_name'], row['artist_location'], row['artist_latitude'], row['artist_longitude']))
    conn.commit()

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

In [13]:
log_files = get_files('../data/log_data')

dfs = []
for file in log_files:
    df = pd.read_json(file, lines=True)
    dfs.append(df)
df2 = pd.concat(dfs, ignore_index=True)
df2

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,1543603476796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8052,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,1543603678796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8053,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,1543603884796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8054,None,Logged In,Rylan,M,61,George,NaN,paid,"Birmingham-Hoover, AL",GET,Downgrade,1.541020e+12,1076,None,200,1543603993796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8056 entries, 0 to 8055
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6820 non-null   object 
 1   auth           8056 non-null   object 
 2   firstName      7770 non-null   object 
 3   gender         7770 non-null   object 
 4   itemInSession  8056 non-null   int64  
 5   lastName       7770 non-null   object 
 6   length         6820 non-null   float64
 7   level          8056 non-null   object 
 8   location       7770 non-null   object 
 9   method         8056 non-null   object 
 10  page           8056 non-null   object 
 11  registration   7770 non-null   float64
 12  sessionId      8056 non-null   int64  
 13  song           6820 non-null   object 
 14  status         8056 non-null   int64  
 15  ts             8056 non-null   int64  
 16  userAgent      7770 non-null   object 
 17  userId         8056 non-null   object 
dtypes: float

In [15]:
df2['userId'].unique()

array([39, 8, 10, 26, 101, 83, 66, 48, 86, 17, 15, 89, 80, 44, 88, 49,
       100, 61, 75, 50, 12, 71, 54, 3, '', '53', '69', '62', '101', '95',
       '10', '15', '63', '49', '6', '52', '99', '43', '25', '51', '26',
       '44', '16', '80', '32', '37', '28', '77', '78', '74', '100', '55',
       '33', '61', '73', '58', '83', '94', '57', '42', '60', '84', '91',
       '24', '97', '75', '35', '81', '27', '29', '12', '66', '88', '50',
       '34', '30', '2', '92', '8', '9', '89', '14', '86', '23', '98',
       '54', '45', '20', '11', '85', '48', '72', '36', '7', '64', '47',
       '67', '13', '18', 96, 6, 16, 52, 37, 69, 32, 74, 7, 18, 36, 14, 35,
       '96', '41', '68', '76', '40', '4', '59', '19', '90', '70', '79',
       '17', '71', '65', '56', '87', '21', '38', '5', '82', '39', '22'],
      dtype=object)

In [16]:
df2['userId'] = df2['userId'].astype(str).str.replace(r'[\D]', '', regex=True)

In [17]:
df2.loc[df2['userId'] == '']

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
186,None,Logged Out,None,None,0,None,NaN,free,None,PUT,Login,NaN,52,None,307,1541207073796,None,
192,None,Logged Out,None,None,0,None,NaN,free,None,GET,Home,NaN,18,None,200,1541239749796,None,
308,None,Logged Out,None,None,3,None,NaN,paid,None,GET,Home,NaN,128,None,200,1541310732796,None,
309,None,Logged Out,None,None,4,None,NaN,paid,None,PUT,Login,NaN,128,None,307,1541310733796,None,
387,None,Logged Out,None,None,0,None,NaN,paid,None,GET,Home,NaN,175,None,200,1541329386796,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7871,None,Logged Out,None,None,1,None,NaN,paid,None,PUT,Login,NaN,977,None,307,1543585407796,None,
7903,None,Logged Out,None,None,15,None,NaN,paid,None,GET,Home,NaN,977,None,200,1543588286796,None,
7904,None,Logged Out,None,None,16,None,NaN,paid,None,PUT,Login,NaN,977,None,307,1543588287796,None,
7926,None,Logged Out,None,None,0,None,NaN,paid,None,PUT,Login,NaN,1097,None,307,1543589944796,None,


In [18]:
df2.loc[df2['userId'] == ''].index

Int64Index([ 186,  192,  308,  309,  387,  408,  419,  420,  444,  481,
            ...
            7801, 7812, 7834, 7835, 7870, 7871, 7903, 7904, 7926, 7996],
           dtype='int64', length=286)

In [19]:
df2.drop(df2.loc[df2['userId'] == ''].index, inplace=True)

## #3: `time` Table

- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `time` table

In [20]:
df2.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [21]:
t = pd.to_datetime(df['ts'], unit='ms')
t

0     2018-11-30 00:22:07.796
1     2018-11-30 01:08:41.796
2     2018-11-30 01:12:48.796
3     2018-11-30 01:17:05.796
4     2018-11-30 01:20:56.796
                ...          
383   2018-11-30 18:44:36.796
384   2018-11-30 18:47:58.796
385   2018-11-30 18:51:24.796
386   2018-11-30 18:53:13.796
387   2018-11-30 19:54:24.796
Name: ts, Length: 388, dtype: datetime64[ns]

In [22]:
time_df = pd.concat([df['ts'], t.dt.hour, t.dt.day, t.dt.weekday, t.dt.month, t.dt.year], axis=1)
time_df.columns = ['unix_timestamp', 'hour', 'day', 'weekday', 'month', 'year']
time_df

,unix_timestamp,hour,day,weekday,month,year
0,1543537327796,0,30,4,11,2018
1,1543540121796,1,30,4,11,2018
2,1543540368796,1,30,4,11,2018
3,1543540625796,1,30,4,11,2018
4,1543540856796,1,30,4,11,2018
...,...,...,...,...,...,...
383,1543603476796,18,30,4,11,2018
384,1543603678796,18,30,4,11,2018
385,1543603884796,18,30,4,11,2018
386,1543603993796,18,30,4,11,2018


In [23]:
# Fix -> ProgrammingError: can't adapt type 'numpy.int64'

import numpy as np
from psycopg2._psycopg import AsIs
from psycopg2.extensions import register_adapter

register_adapter(np.int64, AsIs)

In [24]:
for i, row in time_df.iterrows():
    sql = """
    INSERT INTO time VALUES (%s, %s, %s, %s, %s, %s)
    """
    cur.execute(sql, (row['unix_timestamp'], row['hour'], row['day'], row['weekday'], row['month'], row['year']))
    conn.commit()

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
df2.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [26]:
user_df = df2[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df

,userId,firstName,lastName,gender,level
0,39,Walter,Frye,M,free
1,8,Kaylee,Summers,F,free
2,8,Kaylee,Summers,F,free
3,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
...,...,...,...,...,...
8051,16,Rylan,George,M,paid
8052,16,Rylan,George,M,paid
8053,16,Rylan,George,M,paid
8054,16,Rylan,George,M,paid


In [27]:
for i, row in user_df.iterrows():
    sql = """
    INSERT INTO users VALUES (%s, %s, %s, %s, %s)
    """
    cur.execute(sql, (row['userId'], row['firstName'], row['lastName'], row['gender'], row['level']))
    conn.commit()

## #5: `songplays` Table

Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.

In [28]:
# Need double quotes when Column Name is uppercase letters
sql = """
SELECT songs.id, songs.title, artists.id, artists.name FROM songs 
JOIN artists ON songs."artistID"=artists.id 
WHERE songs.title='Soul Deep' AND artists.name='The Box Tops' AND songs.duration=148.03546;
"""
cur.execute(sql)
result = cur.fetchone()
result

('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 'The Box Tops')

In [29]:
for index, row in df2.iterrows():
    sql = """
    SELECT songs.id, artists.id FROM songs
    JOIN artists ON songs."artistID"=artists.id 
    WHERE songs.title=%s AND artists.name=%s AND songs.duration=%s;
    """
    cur.execute(sql, (row['song'], row['artist'], row['length']))
    result = cur.fetchone()
    
    if result:
        songid, artistid = result
    else:
        songid, artistid = None, None
    sql = """
        INSERT INTO songplays VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    cur.execute(sql, (index, row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'], row['location'], row['userAgent']))
    conn.commit()

In [30]:
conn.close() # Close communication with the database